## Extract, Transform, and Load (ETL) raw data

### Imports and paths

In [2]:
import pandas as pd
from pathlib import Path

RAW_DIR = Path("../data/raw/UCI HAR Dataset")
OUT_PATH = Path("../data/processed/har_clean.csv")
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

RAW_DIR.exists(), OUT_PATH.parent.exists()


(True, True)

### Load feature names and activity labels

In [3]:
# Feature names (561 of them)
features = pd.read_csv(
    RAW_DIR / "features.txt",
    sep=r"\s+",
    header=None,
    names=["idx", "feature"]
)
feature_names = features["feature"].tolist()

# Activity labels (1..6 -> label)
activity_labels = pd.read_csv(
    RAW_DIR / "activity_labels.txt",
    sep=r"\s+",
    header=None,
    names=["code", "activity"]
)
activity_map = dict(zip(activity_labels["code"], activity_labels["activity"]))

len(feature_names), activity_map


(561,
 {1: 'WALKING',
  2: 'WALKING_UPSTAIRS',
  3: 'WALKING_DOWNSTAIRS',
  4: 'SITTING',
  5: 'STANDING',
  6: 'LAYING'})

### Helper function to load train/test split

In [4]:
def load_split(split: str) -> pd.DataFrame:
    """
    split: 'train' or 'test'
    returns df with:
    subject_id, activity_code, activity, split, and 561 feature columns
    """
    X = pd.read_csv(RAW_DIR / split / f"X_{split}.txt", sep=r"\s+", header=None)
    X.columns = feature_names

    y = pd.read_csv(RAW_DIR / split / f"y_{split}.txt", sep=r"\s+", header=None, names=["activity_code"])
    subj = pd.read_csv(RAW_DIR / split / f"subject_{split}.txt", sep=r"\s+", header=None, names=["subject_id"])

    df = pd.concat([subj, y, X], axis=1)
    df["activity"] = df["activity_code"].map(activity_map)
    df["split"] = split
    return df


### Build full dataset and quick checks

In [5]:
train_df = load_split("train")
test_df = load_split("test")

har_df = pd.concat([train_df, test_df], ignore_index=True)

print("Shape:", har_df.shape)  # expect around (10299, 564)
display(har_df[["subject_id", "activity_code", "activity", "split"]].head())

# sanity checks
print("Missing activity labels:", har_df["activity"].isna().sum())
print("Class counts:\n", har_df["activity"].value_counts())


Shape: (10299, 565)


,subject_id,activity_code,activity,split
0,1,5,STANDING,train
1,1,5,STANDING,train
2,1,5,STANDING,train
3,1,5,STANDING,train
4,1,5,STANDING,train


Missing activity labels: 0
Class counts:
 activity
LAYING                1944
STANDING              1906
SITTING               1777
WALKING               1722
WALKING_UPSTAIRS      1544
WALKING_DOWNSTAIRS    1406
Name: count, dtype: int64


### Save processed dataset

In [7]:
har_df.to_csv(OUT_PATH, index=False)